# Selected Economic Characteristics: Health Insurance Coverage from the American Community Survey

**[Work in progress]**

This notebook downloads [selected economic characteristics (DP03)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03) from the American Community Survey 5-Year Data (2009-2018).

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/com.Neo4j.Relate/data/dbmss/dbms-8bf637fc-0d20-4d9f-9c6f-f7e72e92a4da/import


## Download selected variables

* [Selected economic characteristics for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP03 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# HEALTH INSURANCE COVERAGE
             'DP03_0095E': 'civilianNoninstitutionalizedPopulation',
             'DP03_0096E': 'withHealthInsuranceCoverage',
             'DP03_0096PE': 'withHealthInsuranceCoveragePct',
             'DP03_0097E': 'withPrivateHealthInsurance',
             'DP03_0097PE': 'withPrivateHealthInsurancePct',
             'DP03_0098E': 'withPublicCoverage',
             'DP03_0098PE': 'withPublicCoveragePct',
             'DP03_0099E': 'noHealthInsuranceCoverage',
             'DP03_0099PE': 'noHealthInsuranceCoveragePct',
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    if 'Pct' in v:
        print('h.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('h.' + v + ' = toInteger(row.' + v + '),')

h.civilianNoninstitutionalizedPopulation = toInteger(row.civilianNoninstitutionalizedPopulation),
h.withHealthInsuranceCoverage = toInteger(row.withHealthInsuranceCoverage),
h.withHealthInsuranceCoveragePct = toFloat(row.withHealthInsuranceCoveragePct),
h.withPrivateHealthInsurance = toInteger(row.withPrivateHealthInsurance),
h.withPrivateHealthInsurancePct = toFloat(row.withPrivateHealthInsurancePct),
h.withPublicCoverage = toInteger(row.withPublicCoverage),
h.withPublicCoveragePct = toFloat(row.withPublicCoveragePct),
h.noHealthInsuranceCoverage = toInteger(row.noHealthInsuranceCoverage),
h.noHealthInsuranceCoveragePct = toFloat(row.noHealthInsuranceCoveragePct),


In [7]:
print(len(variables.keys()))

9


## Download county-level data using US Census API

In [8]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [9]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,DP03_0095E,DP03_0096E,DP03_0096PE,DP03_0097E,DP03_0097PE,DP03_0098E,DP03_0098PE,DP03_0099E,DP03_0099PE,state,county
1,3245,2949,90.9,2047,63.1,1321,40.7,296,9.1,02,195
2,8189,6463,78.9,1615,19.7,5330,65.1,1726,21.1,02,158
3,4222760,3754381,88.9,2750843,65.1,1432684,33.9,468379,11.1,04,013
4,34003,31115,91.5,19916,58.6,14907,43.8,2888,8.5,04,009


##### Add column names

In [10]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [11]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [12]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [13]:
df.head()

,civilianNoninstitutionalizedPopulation,withHealthInsuranceCoverage,withHealthInsuranceCoveragePct,withPrivateHealthInsurance,withPrivateHealthInsurancePct,withPublicCoverage,withPublicCoveragePct,noHealthInsuranceCoverage,noHealthInsuranceCoveragePct,stateFips,countyFips
1,3245,2949,90.9,2047,63.1,1321,40.7,296,9.1,02,195
2,8189,6463,78.9,1615,19.7,5330,65.1,1726,21.1,02,158
3,4222760,3754381,88.9,2750843,65.1,1432684,33.9,468379,11.1,04,013
4,34003,31115,91.5,19916,58.6,14907,43.8,2888,8.5,04,009
5,46251,40902,88.4,22089,47.8,24658,53.3,5349,11.6,04,023


In [14]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,civilianNoninstitutionalizedPopulation,withHealthInsuranceCoverage,withHealthInsuranceCoveragePct,withPrivateHealthInsurance,withPrivateHealthInsurancePct,withPublicCoverage,withPublicCoveragePct,noHealthInsuranceCoverage,noHealthInsuranceCoveragePct,stateFips,countyFips
897,3204470,2924868,91.3,2185866,68.2,1056014,33.0,279602,8.7,06,073


In [15]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [16]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03HealthInsuranceAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,DP03_0095E,DP03_0096E,DP03_0096PE,DP03_0097E,DP03_0097PE,DP03_0098E,DP03_0098PE,DP03_0099E,DP03_0099PE,state,zip code tabulation area
1,2271,2137,94.1,1560,68.7,902,39.7,134,5.9,51,23833
2,1268,1151,90.8,889,70.1,386,30.4,117,9.2,51,23850
3,13132,12151,92.5,9045,68.9,4918,37.5,981,7.5,51,23851
4,3584,3312,92.4,2507,69.9,1535,42.8,272,7.6,51,23890


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('stateFips')
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,civilianNoninstitutionalizedPopulation,withHealthInsuranceCoverage,withHealthInsuranceCoveragePct,withPrivateHealthInsurance,withPrivateHealthInsurancePct,withPublicCoverage,withPublicCoveragePct,noHealthInsuranceCoverage,noHealthInsuranceCoveragePct,stateFips,postalCode
1,2271,2137,94.1,1560,68.7,902,39.7,134,5.9,51,23833
2,1268,1151,90.8,889,70.1,386,30.4,117,9.2,51,23850
3,13132,12151,92.5,9045,68.9,4918,37.5,981,7.5,51,23851
4,3584,3312,92.4,2507,69.9,1535,42.8,272,7.6,51,23890
5,268,196,73.1,81,30.2,115,42.9,72,26.9,51,23302


In [21]:
# Example data
df.query("postalCode == '90210'")

,civilianNoninstitutionalizedPopulation,withHealthInsuranceCoverage,withHealthInsuranceCoveragePct,withPrivateHealthInsurance,withPrivateHealthInsurancePct,withPublicCoverage,withPublicCoveragePct,noHealthInsuranceCoverage,noHealthInsuranceCoveragePct,stateFips,postalCode
24526,19866,18989,95.6,16129,81.2,6086,30.6,877,4.4,06,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03HealthInsuranceZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [24]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [25]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [26]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [27]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [28]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,civilianNoninstitutionalizedPopulation,withHealthInsuranceCoverage,withHealthInsuranceCoveragePct,withPrivateHealthInsurance,withPrivateHealthInsurancePct,withPublicCoverage,withPublicCoveragePct,noHealthInsuranceCoverage,noHealthInsuranceCoveragePct,stateFips,countyFips,tract,source,aggregationLevel
6,2049,1928,94.1,1801,87.9,173,8.4,121,5.9,06,073,06073008339,American Community Survey 5 year,Tract
7,7016,6782,96.7,5473,78.0,2182,31.1,234,3.3,06,073,06073008347,American Community Survey 5 year,Tract
8,9756,9535,97.7,8266,84.7,2122,21.8,221,2.3,06,073,06073008354,American Community Survey 5 year,Tract
9,6576,5673,86.3,4454,67.7,1728,26.3,903,13.7,06,073,06073008505,American Community Survey 5 year,Tract
10,7371,6869,93.2,5512,74.8,2305,31.3,502,6.8,06,073,06073017604,American Community Survey 5 year,Tract


### Save data

In [29]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03HealthInsuranceTract.csv", index=False)

In [30]:
df.shape

(73056, 14)